**Parte 1**

1. Criar um arquivo JSON, a cada ano, contendo todas as informações importantes dos funcionários.

    - Importando os arquivos, transformando em dicionários, adicionando chaves de Promoção e Demissão e exportando para JSON:

In [4]:
import os
import json
import csv


# Função que transforma um arquivo em dicionário
def dicionario_de_arquivo(ano:int, nome_do_arquivo:str):

    dicionario = {}
    arquivo = str(str(ano)+'/'+nome_do_arquivo)

    if not(os.path.exists(arquivo)):
        print(f'Não existe o arquivo "{nome_do_arquivo}" dentro da pasta {ano}')
        return dicionario

    lista = [linha for linha in (csv.reader(open(arquivo, 'r'), delimiter=',', lineterminator='\n'))]
    dicionario = {chave[0]:{lista[0][i]:chave[i] for i in range(1,len(lista[0]))} for chave in lista[1:]}

    return dicionario


# Função que adiciona a chave de "Demissão" para um dicionário e retorna o dicionário atualizado
def adicionar_demissoes(ano:int, dicionario:dict):

    dicionario_de_demissoes = dicionario_de_arquivo(ano,'demissoes.csv') # criando dicionario de demissoes para usar como base

    if dicionario_de_demissoes == {}:
        return dicionario
    
    ano_min = 2020 # ano inicial do banco de dados, será usado para que as iterações ocorram do ano inserido até o ano_min
    
    for id in dicionario_de_demissoes:
        if id in dicionario:
            {dicionario[id_fun].update({'Demissão':{'Data de Demissão':dicionario_de_demissoes[id_dem].get('Data de Demissão'),'Razão da Demissão':dicionario_de_demissoes[id_dem].get('Razão da Demissão')}}) for id_fun in dicionario for id_dem in dicionario_de_demissoes if id_fun==id_dem}
        else:
            ano_aux = ano-1
            while ano_aux >= ano_min:
                dicionario_aux = dicionario_de_arquivo(ano_aux,'admissoes.csv')
                if id in dicionario_aux:
                    dicionario[id] = dicionario_aux[id]
                    {dicionario[id_fun].update({'Demissão':{'Data de Demissão':dicionario_de_demissoes[id_dem].get('Data de Demissão'),'Razão da Demissão':dicionario_de_demissoes[id_dem].get('Razão da Demissão')}}) for id_fun in dicionario for id_dem in dicionario_de_demissoes if id_fun==id_dem}
                ano_aux -= 1
                
    return dicionario           


# Função que adiciona a chave de "Promoção" para um dicionário e retorna o dicionário atualizado
def adicionar_promocoes(ano:int, dicionario:dict):

    ano_min = 2020 # ano inicial do banco de dados

    for id in dicionario:
            ano_aux = ano-1
            while ano_aux >= ano_min:
                dicionario_aux = dicionario_de_arquivo(ano_aux,'admissoes.csv')
                if id in dicionario_aux:
                    if dicionario[id].get('Cargo') == dicionario_aux[id].get('Cargo'):
                        a = 'Funcionário não promovido, apenas demitido'
                    else:
                        {dicionario[id].update({'Promoção':{'Antigo Cargo':dicionario_aux[id].get('Cargo'),'Antigo Salário':dicionario_aux[id].get('Salário')}})}
                ano_aux -= 1
    return dicionario 


# Função que junta todas as informações de um ano em um único dicionário 
def gerar_dicionario_de_informacoes(ano:int):

    dicionario_ = dicionario_de_arquivo(ano,'admissoes.csv')
    adicionar_promocoes(ano,dicionario_)
    if ano != 2020: 
        adicionar_demissoes(ano,dicionario_)
    
    return dicionario_


# Função que gera o arquivo JSON a partir de um dicionário
def gerar_arquivo_json(ano:int, dicionario:dict):

    nome_do_arquivo = (f'informacoes{ano}.json')
    with open(nome_do_arquivo,'w') as f:
        json.dump(dicionario, f)


# Menu principal para deixar a aplicação mais visual
def menu():

    print('-----------------Bem-vindo(a) ao gerenciador de dados da empresa-----------------',        
          '\n   1. Gerar arquivo de informações para um ano específico',
          '\n   2. Gerar arquivo de demissões',
          '\n   3. Identificar penetras na festa',
          '\n   4. Sair',
          )
    opcao = input('Digite a opção escolhida (1, 2, 3 ou 4): ')
    while opcao != '4':
        if opcao == '1':
            ano = int(input('Digite o ano que deseja gerar o arquivo (2020, 2021 ou 2022): '))
            if ano != 2020 and ano != 2021 and ano != 2022:
                print('Ano inválido. Digite um ano entre 2020 e 2022')
            else:
                dicionario_final = gerar_dicionario_de_informacoes(ano)
                gerar_arquivo_json(ano,dicionario_final)
        opcao = input('Para continuar o gerenciamento digite uma das opções (1, 2 ou 3), para sair do sistema digite 4: ')


menu()

-----------------Bem-vindo(a) ao gerenciador de dados da empresa----------------- 
   1. Gerar arquivo de informações para um ano específico 
   2. Gerar arquivo de demissões 
   3. Identificar penetras na festa 
   4. Sair
